1) Fasta seq-es parsing

In [3]:
import fastaparser
import os

In [26]:
def fasta_files_to_dict(fasta_files_dir):
    fasta_dict = {}
    fasta_files = os.listdir(fasta_files_dir)
    fasta_files.sort()
    for file in fasta_files:
        with open(fasta_files_dir + '/' + file) as fa:
            parser = fastaparser.Reader(fa, parse_method='quick')
            for item in parser:
                fasta_dict[item.header.split()[0][1:]] = item.sequence
    return fasta_dict

In [27]:
fasta_files_dir = '/home/doctor/pompan/cluster_extraction/input/proteins'

In [28]:
big_dic = fasta_files_to_dict(fasta_files_dir)

2) TSV parsing

In [19]:
def tsv_clusters_parser(single_copy_orthologs):
    '''
    Args:
        single_copy_orthologs: filtered tsv file with clusters of orthologs (proteinOrtho output file, where clusters with exceeding or deficient number of genes compared to the total number of observed sequences are deleted)
    Returns:
        list of clusters, where each cluster is a list of names, representing sequences included in that cluster
    '''
    with open(single_copy_orthologs) as file:
        clusters = file.read().rstrip().split('\n')
    result = []
    for cluster in clusters:
        result += [cluster.split('\t')[3:]]
    return result

In [20]:
single_copy_orthologs = '/home/doctor/pompan/cluster_extraction/single_copy_orthologs.tsv'

In [21]:
result = tsv_clusters_parser(single_copy_orthologs)

In [22]:
for lst in result:
    print(lst)

['HAKLFBJG_00008', 'EKGDDEJB_00009', 'LLILAMIN_00006', 'JALFEBEM_00008', 'NDMFMGIM_00007', 'EJIIHJBD_00008', 'LPALIGJD_00006']
['HAKLFBJG_00001', 'EKGDDEJB_00001', 'LLILAMIN_00001', 'JALFEBEM_00001', 'NDMFMGIM_00001', 'EJIIHJBD_00001', 'LPALIGJD_00001']
['HAKLFBJG_00003', 'EKGDDEJB_00003', 'LLILAMIN_00003', 'JALFEBEM_00003', 'NDMFMGIM_00003', 'EJIIHJBD_00003', 'LPALIGJD_00003']
['HAKLFBJG_00006', 'EKGDDEJB_00007', 'LLILAMIN_00004', 'JALFEBEM_00006', 'NDMFMGIM_00005', 'EJIIHJBD_00006', 'LPALIGJD_00004']
['HAKLFBJG_00002', 'EKGDDEJB_00002', 'LLILAMIN_00002', 'JALFEBEM_00002', 'NDMFMGIM_00002', 'EJIIHJBD_00002', 'LPALIGJD_00002']


3) Extraction itself

In [23]:
def extract_clusters(fasta_dict, clusters_list):
    list_of_cluster_dicts = []
    for cluster in clusters_list:
        cluster_dict = {}
        for seq_name in cluster:
            cluster_dict[seq_name] = fasta_dict[seq_name]
        list_of_cluster_dicts += [cluster_dict]
    return list_of_cluster_dicts

In [31]:
list_of_cluster_dicts = extract_clusters(big_dic, result)

4) Make output fasta files

In [46]:
def fasta_writer(fasta_dict, output_path): #Method defined in other scripts
    """
    Args:
        fasta_dict: dictionary with key == fasta header; value == concatenated sequences
        output_path: path to the file for output
    Returns:
        writes concatenated sequences to the output file in fasta format
    """
    file = output_path
    with open(file, "w") as fh:
        for key, value in fasta_dict.items():
            fh.write('>' + key + '\n' + value + '\n')

In [47]:
def make_fasta_files(list_of_cluster_dicts, output_dir):
    count = 0
    for cluster_dict in list_of_cluster_dicts:
        count += 1
        fasta_writer(cluster_dict, output_dir + '/cluster' + str(count))

In [48]:
output_dir = '/home/doctor/pompan/cluster_extraction/output'

In [49]:
make_fasta_files(list_of_cluster_dicts, output_dir)